In [1]:
X = []
Y = []

for line in open('train_data.txt').readlines():
    line = line.strip()
    a = line.index('[')
    b = line.index(']')
    grid = list(map(int, line[a+1: b].split(', ')))
    result = int(line[b+1:].strip())
    X.append(grid)
    Y.append((result == 0, result == 1))

In [2]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

X = np.array(X)
Y = np.array(Y)

pca = PCA(n_components=7)
X = pca.fit_transform(X)  # the device only has 7 cores so we should use PCA to reduce the dimensions

scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X)

In [3]:
from sklearn.model_selection import train_test_split


train_features, test_features, train_labels, test_labels = train_test_split(
    X, Y, train_size=0.7, shuffle=True
)

In [4]:
# number of qubits is equal to the number of features
num_qubits = 2

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 2000

In [5]:
from qiskit import *
IBMQ.providers()
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import ZFeatureMap
from qiskit_machine_learning.algorithms import PegasosQSVC


from qiskit.visualization import plot_histogram

IBMQ.save_account('53fbe5f38c448c1c5d19a96fa47bd10ee765b1a9721453022212a2f5b3b29c0fb75a665962ef38dd5fde63cd26ddb2cd9d86fe78c9a37b947ff91e6bf911b0ea', overwrite=True)

IBMQ.load_account()

provider = IBMQ.get_provider(hub = 'ibm-q')

# devices = provider.backends()
# for _device in devices:
#     print(_device.name(), _device.configuration().n_qubits)
#     if _device.configuration().n_qubits >= 9:
#         device = _device
# device = provider.get_backend('ibm_oslo')


from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=10598, seed_transpiler=10598)

In [6]:
from qiskit_machine_learning.kernels import QuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)
# pegasos_backend = QuantumInstance(backend = device)
qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
pegasos_qsvc0 = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)
pegasos_qsvc0.fit(train_features, train_labels[:, 0])


feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)
# pegasos_backend = QuantumInstance(backend = device)
qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
pegasos_qsvc1 = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)
pegasos_qsvc1.fit(train_features, train_labels[:, 1])

In [7]:
# testing
pred1 = np.array(pegasos_qsvc1.predict(test_features[:200]))
pred0 = np.array(pegasos_qsvc0.predict(test_features[:200]))
y1 = np.array(test_labels[:200, 1])
y0 = np.array(test_labels[:200, 0])

print(f"PegasosQSVC01 classification test score: {sum(pred1 == y1) / len(pred1)}")

pegasos_score1 = pegasos_qsvc1.score(test_features[:200], test_labels[:200, 0])
print(f"PegasosQSVC1 classification test score: {sum(pred0 == y0) / len(pred0)}")

print(f"both test score: {sum((pred1 == y1) & (pred0 == y0)) / len(pred1)}")

PegasosQSVC01 classification test score: 0.675
PegasosQSVC1 classification test score: 0.585
both test score: 0.45


In [8]:
for a, b in zip(pegasos_qsvc1.predict(test_features[:30]), test_labels[:30, 0]):
    print(a, b)

True True
False True
False True
False True
True False
True False
True True
True True
True False
True False
False True
False False
True True
False True
True False
True False
True False
True False
True True
True False
True False
False True
True False
True False
True True
True False
False True
True True
True False
True False


In [9]:
import pickle
from sklearn.pipeline import Pipeline
from wrapper import Wrapper

pipe0 = Pipeline([('dim-reduction', pca), ('scaler', scaler), ('qsvc', pegasos_qsvc0)])
pipe1 = Pipeline([('dim-reduction', pca), ('scaler', scaler), ('qsvc', pegasos_qsvc1)])

pickle.dump(Wrapper(pipe0=pipe0, pipe1=pipe1), open('quantum-svc-model.pkl', 'wb'))
pickle.dump(pipe1, open('quantum-svc-model1.pkl', 'wb'))